# scRNA-seq notes

10x PBMC data are hosted in https://s3-us-west-2.amazonaws.com/10x.files/samples/cell/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz

## scRNA-seq

Single-cell RNA sequencing (scRNA-seq) is a great tool to allow us to see at cellular resolution that you can't get using bulk RNA-seq which only measures average expression levels across all cells. scRNA-seq can estimate the distribution of expression levels across the cells in your sample. It also has the advantage of sampling all transcripts in your sample in an unbiased way, unlike microarrays and RT-qPCR which are limited to a limited set of transcripts. This means that scRNA-seq is great for:
- Identifying rare cell types
- Studying cell differentiation during development
- Building gene atlases

**scRNA-seq protocols**

There are many scRNA-seq protocols available and the dataset sizes from these are continually growing:

<img src="imgs/scRNA-seq_protocols.png" width = 600>

*Chen, Teichman and Meyer, 2018. Abbreviations: cDNA, complementary DNA; DNA pol I, DNA polymerase I; FACS, fluorescence-activated cell sorting; PCR, polymerase chain reaction; RNase H, ribonuclease H; RT, reverse transcription; TSO, template-switching oligonucleotide*

Protocols usually follow these steps:
1. Cells in a solid sample are dissociated
2. (optional: cells can be selected for using markers, fluorescent transgenes or staining dyes)
3. They are then isolated and captured. Three methods for this are:
    - **Microtitre-plate-based methods** - cells are put in individual wells using things like pipetting, microdissection of fluorescent activated cell sorting (FACS). This method allows you to take pictures of the cells so you can remove damage cells or wells that have more than one cell (known as doublets). FACS also lets you sort cells by things like cell size or intensity of labels and place them in specific wells to give you some more infomation for your downstream analysis. However, this method is work-intestive so is low-throughput.
    - **Microfluidic-array-based methods** - these are efficient methods that combines cell capture with library preparation so is high-throughput. However, it only captures around 10% of cells so is not good for rare cell types or small samples. Also, the nanowells used are customised for particular sizes which can introduce some bias when studying tissues.
    - **Microgluidic-droplet-based methods** - these are the most popular methods used today and have the highest through-put. Cells are trapped in an nanoliter-sized oil droplet with a bead. This bead has the things needed to make the library and a unique barcode that is attached to all reads from one cell. This means that sequencing can be done on all cells together.
4. RNA is extracted
5. Then it is reverse-transcribed into cDNA
6. cDNA is then amplified (by *in vitro* transcription or PCR)
7. This is then built into a sequencing library
8. Sequencing using NGS
9. An expression matrix can then be used for further analysis

<img src="imgs/scRNA-seq_workflow.png" width = 600>

*Taken from wikipedia https://en.wikipedia.org/wiki/Single_cell_sequencing*

The protocol you choose to use will depend on study design and aims. Different protocols can have variations in accuracy and sensitivity (how much starting RNA is needed). There have been studies comparing different protocols: Ziegenhaln et al, 2017; Scensson et al, 2017.

**Drawbacks**

There are a few problems that come with scRNA-seq:
- Low amount of RNA from cells can cause complications:
    - **Amplification** is needed which can lead to biases in the data.
    - **'Dropouts'** occur when some tramscripts are not captured before amplification. It means some cells this gene will appear to have no expression so data will have many zeros.
- Unlike bulk RNA-seq, it is difficult to have replicates as the libraries represent a single cell, rather than a cell population, so distinguishing technical noise and biological variability is tricky. However, two methods can be used to do this:
    - **Spike-ins** - this allows you to normalise the data by adding in synthetic RNA molecules at a known concentration. The most commonly used are a set of 96 spikes from the External RNA Control Consortium (ERCC).
    - **UMIs** - these unique molecular identifiers are 4-20 bp barcodes added to the 3' or 5' end of each transcript before amplification which is followed by targeted sequencing of the 3' or 5' end of the transcripts. This allows you to quantify the number of transcripts before amplification.
- Batch effects caused by confounding variables when different samples are processed seperately

53:00 - scRNA-seq packages
54:46 - quantification

## Processing raw scRNA-seq data to make a Count Matrix

### Method for protocols other than 10x Chromium

The output of scRNA-seq is usually in the form of **FastQ** files which have this format:

>\>ReadID <br>
>CDNA READ SEQUENCE
>    
>+
>
>SEQUENCING QUALITY SCORES

Protocols that use paired-end sequencing may have barcodes on one or both reads. When UMIs are used, one read will have the cell and UMI barcodes, while the other will contain the transcript sequence. 

**Read quality**

The first step after getting the FastQ file is to check the quality of the reads. Before we get this file, reads of low quality will have been removed and the data will have been normalised to account for amplification. Checking read quality can be done using a tool call FastQC which can be used for both bulk and single-cell RNA-seq data. FastQC outputs a report on the read quality using the quality scores in the FastQ file.

https://www.bioinformatics.babraham.ac.uk/projects/fastqc/

Run FastQC using the following command:

`fastqc -o output_directory fastQ_file`

This will prodyce a zip and html file for each of the paired-end reads. The report is in the html file. 

<img src="imgs/FastQC.png" width = 600>

*Taken from https://www.singlecellcourse.org/*

**Trimming reads**

Next, sequencing adapters need to be trimmed off reads. Sequencing also tends to have low quality regions at the ends and poly-A sequence, so these can be trimmed too. This can be done using Trim Galore! which is a wrapper for the reads trimming software cutadapt. If UMIs are used, it is not necessary to trim the read with the UMI and cell barcode, just the one with the transcript. 
<img src="imgs/UMIs.png" width = 400>

*Adapted from https://www.singlecellcourse.org/*

This can be run using the following command:

`trim_galore --nextera -o output_directory FastQ_file`

After this is done, another FastQC report should be made to check the read quality again.

**Demultiplexing**

Demultiplexing is the process of grouping all the reads per cell by combining all the reads with the same cell barcode (and then removing the barcodes) and is done differently depending on the protocol used. The tool that can be applied to most protocols is zUMIs which can be used for most UMI-based protocols. 

**Read alignment**

The next step is to find out which genes our reads correspond to. We do this by mapping them to a reference genome. There are many tools for this. If spike-ins are used, the reference should be changed with the DNA sequence of the spike-in first. UMIs should also be used, these can be added to the read name. Also, before aligning, cells should be checked to see if they have enough mappable reads (~60-70% - but it is important to compare with other cells in the sample and remove outliers). Cells with a low proportion of mappable reads could indicate contamination and should be removed. Below are two:

**STAR** - this tool tries to find the longest possible seuence whihc matches the sequences in the reference genome. It is able to find regions where the read spans more than one exon making it a 'splice aware' aligner. This means it can be used to xtudy alternative splicing or chromosomal rearragements. 

<img src="imgs/STAR.png" width = 400>

*Dobin et al, 2013*

You need to provide the reference genome (**FASTA file**) and annotations (GTF) which STAR uses to make a genome index. STAR then aligns the reads to this. Run STAR using this command:

`STAR --runThreadN 4 --runMode genomeGenerate --genomeDir output_directory --genomeFastaFiles reference_fasta_file`

The output is a **BAM file** which stores the mapped reads. The human-readable version is called a SAM file. It includes information like the mapping quality and the read's position. BAM files can be viewd using IGV. Sometimes sequencing methods provide BAM files only; therefore, these will need to be converted to FastQ files using bedtools/Picard to perform the QC. CRAM files are similar to BAM files but are more compressed and can be converted to BAM files using samtools. 

**Kallisto** - this is a **pseudo-aligner** which maps k-mers, rather than reads, to a reference genome. K-mers are sequences of k length from a read. Psuedo-alignment is much faster than alignment and may be more robust to sequencing errors as it would only affect some, but not all, of the k-mers of a read. Kallisto, unlike STAR, maps to a reference transcriptome, rather than a reference genome meaning they map to isoforms rather than genes. This is difficult because:
- scRNA-seq has lower coverage than bulk RNA-seq
- Many protocols have a 3' coverage bias, so it is hard to tell which isoform a read came from if they that differ at the 5' end.
- Some protocols have short read lengths which also make it challenging to tell which isoform a read came from

Kallisto mitigates this by assigning reads that could map to multiple isoforms to an equvilance class so downstream analyses like clusering use this instead. 

**Alignment QC** - After the reads have been mapped to the genome, the quality of the mapping needs to be evaluated. Tools like RSeQC can be used. As before, look at the whole dataset and remove outliers. It can be done by looking at read depth along the transcripts as well as the proportion of:
- Reads mapping to rRNA/tRNA
- Uniquely mapping reads
- Reads mapping over splice juntions

<img src="imgs/gene_coverage.png" width = 300>

*Taken from https://www.singlecellcourse.org/*

**Reads quantification**

Next, we need to quantify the expression level of each gene for each cell. For non-UMI scRNA-seq data, tools for bulk RNA-seq can be used like HT-seq and FeatureCounts.

scRNA-seq data that do include UMIs

31:45 https://www.youtube.com/watch?v=IrlNcJwPClQ

1:15